In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def binarycode_to_signal(bitstream, step):
    T = 1  # assume the pulse period is 1
    A = 1  # assume the amplitude is 1
    pulse = np.ones(math.ceil(T/step)) * A
    # pulse = pulse*A
    signal = np.zeros(len(bitstream)*len(pulse))
    # Polar nonreturn to zero
    for i in range(len(bitstream)):
        if bitstream[i] == 1:
            signal[i*len(pulse):(i+1)*len(pulse)] = 1*pulse   
        else:
            signal[i*len(pulse):(i+1)*len(pulse)] = -1*pulse
    return signal  

In [ ]:
def add_noise(generated_bits, sigma,n,step):
    generated_noise = np.random.normal(loc=0, scale=sigma, size=n*math.ceil(1/step))

    scaled_samples_with_noise = np.ones((generated_bits.shape[0], math.ceil(1/step)))
    scaled_samples = np.ones((generated_bits.shape[0], math.ceil(1/step)))
    for i in range(generated_bits.shape[0]):
        scaled_samples[i, :] *= generated_bits[i]
        scaled_samples_with_noise[i, :] *= generated_bits[i]
        scaled_samples_with_noise[i, :] += generated_noise[i* math.ceil(1/step):(i+1)* math.ceil(1/step)]
    return scaled_samples, scaled_samples_with_noise

In [ ]:
def calc_c(scaled_samples_with_noise, received_filter,num_of_bits,num_of_samples_per_bit):
    convolution_result_sampled_Tp = np.zeros(num_of_bits)
    
    if(received_filter is None):
        convolution_result = scaled_samples_with_noise.flatten()
    else:
        convolution_result = np.convolve(scaled_samples_with_noise.flatten(), received_filter)
    for i in range(num_of_bits):
        convolution_result_sampled_Tp[i] = convolution_result[(num_of_samples_per_bit - 1) + num_of_samples_per_bit * i]

    return convolution_result, convolution_result_sampled_Tp

In [ ]:
def calc_simulated_BER(true_bits, convolution_result):
    # applying lamda thresholed (lamda optimum = 0)
    received_samples = np.ones(true_bits.shape[0])
    received_samples += (-2 * (convolution_result < 0))

    # calculate probability of error
    error_probability = np.sum(received_samples != true_bits)
    error_probability /= true_bits.shape[0]
    return error_probability


def calc_theoretical_BER(erfc_parameter,factor):
    return factor*math.erfc(erfc_parameter)# Question multiply with half